In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
google_api_key = os.getenv('GOOGLE_API_KEY')
if not google_api_key: 
    raise ValueError("GOOGLE_API_KEY is not set in the environment variables.")

In [2]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [4]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model=llm,  
    tools=[get_weather],  
    prompt="You are a helpful assistant"  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in San Francisco?"}]}
)

{'messages': [HumanMessage(content='what is the weather in San Francisco?', additional_kwargs={}, response_metadata={}, id='5a3c220b-5ebd-4e61-a53d-98f726ddd758'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e8cf91ad-75e0-4612-94b9-fdd2813cb93f-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'fac86331-a59c-415a-afa3-4ff84ac8237a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 27, 'output_tokens': 6, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in San Francisco!", name='get_weather', id='11972c46-3688-4a34-9672-57a85a1b73b2', tool_call_id='fac86331-a59c-415a-afa3-4ff84ac8237a'),
  AIMessage(content="It's always sunny in San Francisco!"

In [7]:
agent = create_react_agent(
    model=llm,  
    tools=[get_weather],  
    prompt="Never answer questions about the weather."  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in San Francisco?"}]}
)

{'messages': [HumanMessage(content='what is the weather in San Francisco?', additional_kwargs={}, response_metadata={}, id='2a923cae-932e-4ed3-ae41-e8ad0125c34b'),
  AIMessage(content='I am sorry, I cannot fulfill this request. The available tools lack the desired functionality.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e66707a7-1663-4041-9ee3-0e92bd790da3-0', usage_metadata={'input_tokens': 29, 'output_tokens': 19, 'total_tokens': 48, 'input_token_details': {'cache_read': 0}})]}

In [8]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    checkpointer=checkpointer  
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in san francisco?"}]},
    config  
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    config
)

In [9]:
sf_response, ny_response

({'messages': [HumanMessage(content='what is the weather in san francisco?', additional_kwargs={}, response_metadata={}, id='4ad41362-e4cb-447c-a314-de69fe4eb976'),
   AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "san francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--086e55c6-3b32-41b7-9cc1-bf97b97839b3-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'san francisco'}, 'id': '1af89559-6449-4165-92c6-3a5a086dc331', 'type': 'tool_call'}], usage_metadata={'input_tokens': 22, 'output_tokens': 6, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}}),
   ToolMessage(content="It's always sunny in san francisco!", name='get_weather', id='1326fce0-6577-4b49-bd63-c43acf48b493', tool_call_id='1af89559-6449-4165-92c6-3a5a086dc331'),
   AIMessage(content="It's always sunny in San Francis

In [15]:
from pydantic import BaseModel

class WeatherResponse(BaseModel):
    city: str
    conditions: str

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    response_format=WeatherResponse  
)
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in san francisco?"}]}
)

response["structured_response"]

WeatherResponse(city='San Francisco', conditions='Sunny')